## TODOs:
* Rename func_order to lf_order

In [1]:
from fastai2.text.all import *
from lazylabel.text.all import *
# from fastai2.basics import *

In [2]:
source = untar_data(URLs.IMDB_SAMPLE)

In [3]:
df = pd.read_csv(source/'texts.csv')
df.head(1)

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False


In [4]:
df['text2'] = df['text']

In [5]:
ABSTAIN,POS,NEG = 'abstain','positive','negative'

In [6]:
vocab = [ABSTAIN,POS,NEG]

In [7]:
labeller = Labeller()
col_reader,tokenizer,numericalize = mk_transform(ColReader('text2')),Tokenizer.from_df('text'),Numericalize

In [8]:
def liked(x): return POS if 'liked' in x else ABSTAIN
def hated(x): return NEG if 'hated' in x else ABSTAIN
def love_string(x): return POS if 'love' in x else ABSTAIN
labeller.register_funcs(col_reader, [liked,hated,love_string])

In [9]:
@labeller(tokenizer)
def love(x): return POS if 'love' in x else ABSTAIN

In [10]:
splits = ColSplitter()(df)

In [12]:
#TODO: How to not pass ABSTAIN?
# Adds option to cache results, should also fix #1
tls = TfmdLists(df, [col_reader,tokenizer,numericalize])
tasks = tasks_labels(labeller, tls, vocab, splits=splits)

In [14]:
true_lbls = TfmdLists(df, [ColReader('label'), Categorize(vocab=vocab)], splits=splits)

In [15]:
dset_train = Datasets(tls=[tasks]).train
dset_valid = Datasets(tls=[tasks,true_lbls]).valid

In [16]:
dls = DataLoaders.from_dsets(dset_train, dset_valid, drop_last=False)

In [17]:
# TODO: Hardcoded abstain as 0, but should be a variable
def acc(pred,targ):
    mask = pred!=0
    if mask.nonzero().numel() > 0:
        return (pred[mask]==targ[mask]).float().mean()
    return 0.

In [18]:
learn = Learner(dls, MajorityLabelVoter(), loss_func=lambda *_: tensor(0.), metrics=acc)

In [19]:
learn.validate()

(#2) [0.0,0.5182222127914429]

In [20]:
labeller.summary(dls.train)

,Coverage,Polarity
liked,6.12% (49),1
hated,1.00% (8),1
love_string,23.12% (185),1
love,17.00% (136),1


In [21]:
labeller.summary(dls.valid)

,Coverage,Polarity,Accuracy,Correct,Incorrect
liked,4.50% (9),1,0.222222,2,7
hated,1.00% (2),1,1.000000,2,0
love_string,23.50% (47),1,0.553191,26,21
love,14.50% (29),1,0.482759,14,15


In [23]:
labeller.find(dls.train, vocab, ['love', 'hated'], [POS, NEG])

[304, 695]

In [24]:
tls.decode(tls[695])

"xxbos h xxrep 3 m , yeah this episode is extremely underrated . \n\n xxmaj even though there is a xxup lot of bad writing and acting at parts . i think the good over wins the bad . \n\n i love the xxunk parts and the big ' twist ' at the end . i absolutely love that scene when xxmaj michelle xxunk xxmaj tony . xxmaj it 's actually one of my favorite scenes of xxmaj season 1 . \n\n xxmaj for some reason , people have always hated the xxmaj xxunk episodes , yet i have always liked them . xxmaj they 're not the best , in terms of writing . but the theme really does interest me , \n\n xxmaj i 'm gon na give it a xxup three star , but if the writing were a little more consistent i 'd give it xxup four ."